In [1]:
# Some basic imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import warnings
import ast
import re

In [2]:
# Load the general data:
success_movies_path = 'data/processed_data/success_movies.csv'
ethnicity_path = 'data/processed_data/diversity_final.csv'

success_movies_df = pd.read_csv(success_movies_path, sep=',')
ethnicity_df = pd.read_csv(ethnicity_path, sep=',')
success_movies_df.head()

,Movie_name,Movie_release_date,Ratings,Wikipedia_movie_ID,Actor_ethnicity,Movie_countries,Movie_box_office_revenue,Nomination,Success
0,$,1971,6.3,4213160.0,Hungarian Americans,{'United States of America'},NaN,False,0
1,$9.99,2008,6.7,20624798.0,Australians,"{'Australia', 'Israel'}",NaN,False,0
2,'R Xmas,2001,5.7,24799628.0,Italian Americans,"{'France', 'United States of America'}",NaN,False,0
3,'Til There Was You,1997,4.8,17841580.0,Irish Americans,{'United States of America'},NaN,False,0
4,'Til We Meet Again,1940,6.7,16038391.0,Irish Americans,{'United States of America'},NaN,False,0


## Assessing Correlation between Diversity and Success

- Correlation Analysis: Evaluate the dependency between diversity and film success using: Pearson correlation (stats.pearson) for linear associations.

- Spearman correlation (stats.spearman) 

- Regression Analysis: Investigate the association type (e.g., weak, negative, positive) through linear regression.

In [3]:

len(ethnicity_df['Wikipedia_movie_ID'].unique())

32510

In [5]:
len(success_movies_df['Wikipedia_movie_ID'].unique())

21090

In [6]:
len(success_movies_df)

21481

In [7]:
success_movies_df.value_counts()

Movie_name               Movie_release_date  Ratings  Wikipedia_movie_ID  Actor_ethnicity    Movie_countries                                             Movie_box_office_revenue  Nomination  Success
Emma                     1996                6.6      2654649.0           English people     {'United Kingdom'}                                          2500000.0                 True        0          3
Contact                  1997                7.5      422269.0            English Americans  {'United States of America'}                                3255426.0                 True        0          2
Othello                  1965                7.0      1345984.0           English people     {'United Kingdom'}                                          3400000.0                 True        0          2
Pinocchio                1940                7.5      187010.0            Jewish people      {'United States of America'}                                7856048.0                 True      